In [1]:
import tensorflow as tf
import numpy as np

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#train 데이터
xy = np.loadtxt('train_0.csv', delimiter=',', dtype = np.float32)
x_data = xy[:, :297]
print(np.shape(x_data))
y_data =xy[:, 297:300]
y_change = (xy[:, 299] - xy[:, 297]) / xy[:, 299] * 100
y_change = np.vstack(y_change)
print(np.shape(y_change))

#test 데이터
xy_test = np.loadtxt('train_1.csv', delimiter=',', dtype = np.float32)
x_test = xy_test[:, :297]
print(np.shape(x_test))
y_test_data = xy_test[:, :297]
y_test = (xy_test[:, 299] - xy_test[:, 297]) / xy_test[:, 299] * 100
y_test = np.vstack(y_test)
print(np.shape(y_test))

(59, 297)
(59, 1)
(31, 297)
(31, 1)


In [3]:
nb_classes = 10

x = tf.placeholder(tf.float32, shape = [None, 297])
y = tf.placeholder(tf.float32, shape = [None, 1])

w1 = tf.get_variable('weight1', shape=[297,256], initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]), name = 'bias1')
_layer1 = tf.nn.relu(tf.matmul(x,w1) + b1)

w2 = tf.get_variable('weight2', shape=[256,256], initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]), name = 'bias2')
_layer2 = tf.nn.relu(tf.matmul(_layer1,w2) + b2)

w3 = tf.get_variable('weight3', shape=[256,128], initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]), name = 'bias3')
_layer3 = tf.nn.relu(tf.matmul(_layer2,w3) + b3)

w4 = tf.get_variable('weight4', shape=[128,1], initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1]), name = 'bias4')
hypothesis = tf.matmul(_layer3,w4) + b4
cost = tf.reduce_mean(tf.square(hypothesis - y_change))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.1)
train = optimizer.minimize(cost)

count = 0
for i in range(np.shape(y_test)[0]):   
    if tf.cast(hypothesis, tf.int32)[i] == tf.cast(y_test, tf.int32)[i]:
        count += 1
accuracy = count/ np.shape(y_test)[0]

training_epochs = 201

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(training_epochs):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(cost), feed_dict = {x: x_data, y: y_change})
        print(sess.run(accuray), feed_dict = {x: x_test, y: y_test})


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,297]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,297], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "C:\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "C:\Anaconda3\lib\asyncio\base_events.py", line 422, in run_fore